In [72]:
import os
import re
import pandas as pd
from IPython.display import display, Latex

In [73]:
# get the names of each file that starts with 'bench' in the current directory
files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('bench')]
compute_methods = [f[6:-4] for f in files]
# replace underscores with spaces
compute_methods = [re.sub('_', ' ', f) for f in compute_methods]
compute_methods

['eigen', 'eigen approx', 'stl', 'stl approx', 'xsimd']

In [74]:
all_data = []

# Process each file and append data to the list
for file_name, compute_method in zip(files, compute_methods):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            # Match the benchmark line
            if 'Benchmarking' in lines[i]:
                layer_match = re.search(r'Benchmarking (\w+) layer, with input size (\d+) and output size (\d+), with signal length (\d+) seconds', lines[i])
                if layer_match:
                    layer_type = layer_match.group(1)
                    input_size = int(layer_match.group(2))
                    output_size = int(layer_match.group(3))
                    seconds = int(layer_match.group(4))

                    # Process the next lines to get processing time and real-time factor
                    i += 2  # Skip the 'RTNEURAL DYNAMIC...' line
                    processing_time_match = re.search(r'Processed \d+ seconds of signal in (\d+\.\d+) seconds', lines[i])
                    if processing_time_match:
                        processing_time = float(processing_time_match.group(1))
                    
                    i += 1
                    real_time_factor_match = re.search(r'(\d+\.\d+)x real-time', lines[i])
                    if real_time_factor_match:
                        real_time_factor = float(real_time_factor_match.group(1))
                    
                    # Append the collected data to the list, including the compute method
                    all_data.append([layer_type, input_size, output_size, seconds, processing_time, real_time_factor, compute_method])
            
            i += 1

# Create DataFrame
df = pd.DataFrame(all_data, columns=['Layer Type', 'Input Size', 'Output Size', 'Seconds', 'Wall Time', 'Real-Time Factor', 'Compute Method'])

# Display DataFrame
df

,Layer Type,Input Size,Output Size,Seconds,Wall Time,Real-Time Factor,Compute Method
0,dense,4,4,4,0.008347,479.190,eigen
1,dense,8,8,4,0.005731,697.963,eigen
2,dense,16,16,4,0.004573,874.795,eigen
3,dense,32,32,4,0.008962,446.329,eigen
4,dense,64,64,4,0.032131,124.490,eigen
...,...,...,...,...,...,...,...
240,sigmoid,16,16,32,0.022705,1409.350,xsimd
241,sigmoid,32,32,32,0.043485,735.885,xsimd
242,sigmoid,64,64,32,0.080985,395.137,xsimd
243,sigmoid,128,128,32,0.156999,203.823,xsimd


In [75]:
pivot_df = df.pivot_table(index=['Layer Type', 'Compute Method'],
                          columns='Input Size',
                          values='Wall Time',
                          aggfunc='first')  # Using 'first' here as a placeholder; no aggregation is actually needed.


pivot_df

Input Size                      4         8         16        32         64   \
Layer Type Compute Method                                                      
conv1d     eigen           0.004805  0.012126  0.064102  0.148811   1.407740   
           eigen approx    0.011008  0.017569  0.049356  0.101118   0.908134   
           stl             0.007266  0.054377  0.653435  1.759890  15.142400   
           stl approx      0.012041  0.042484  0.435642  1.109390   9.020560   
           xsimd           0.010408  0.062329  0.402620  0.556938   2.716340   
dense      eigen           0.008347  0.005731  0.004573  0.008962   0.032131   
           eigen approx    0.006982  0.005431  0.004352  0.008866   0.031640   
           stl             0.005169  0.008346  0.019095  0.089807   0.499090   
           stl approx      0.005156  0.008075  0.018986  0.090697   0.509343   
           xsimd           0.008295  0.010002  0.018357  0.049436   0.158758   
gru        eigen           0.014499  0.016987  0.026294  0.062539   0.211060   
           eigen approx    0.012815  0.014715  0.020283  0.052111   0.191262   
           stl             0.033454  0.089232  0.266512  0.894249   3.920630   
           stl approx      0.014454  0.036857  0.125083  0.602819   3.436030   
           xsimd           0.032143  0.057760  0.128522  0.339466   1.028440   
lstm       eigen           0.012299  0.015120  0.029608  0.083030   0.354849   
           eigen approx    0.011902  0.015063  0.021791  0.070306   0.324553   
           stl             0.040875  0.136037  0.418509  1.376620   5.645630   
           stl approx      0.019541  0.062217  0.207566  0.879157   4.721530   
           xsimd           0.038011  0.072747  0.168950  0.444768   1.428400   
relu       eigen           0.008118  0.009631  0.010882  0.017403   0.029641   
           eigen approx    0.008442  0.010165  0.011556  0.018900   0.032514   
           stl             0.010895  0.015798  0.029106  0.053231   0.109398   
           stl approx      0.010161  0.014874  0.027287  0.050930   0.105940   
           xsimd           0.004772  0.005501  0.006539  0.011788   0.025532   
sigmoid    eigen           0.013573  0.018722  0.025987  0.044098   0.084852   
           eigen approx    0.011123  0.017783  0.011890  0.021072   0.041867   
           stl             0.048567  0.099441  0.193933  0.383510   0.748444   
           stl approx      0.011335  0.020355  0.036412  0.072870   0.147187   
           xsimd           0.010124  0.014087  0.022705  0.043485   0.080985   
tanh       eigen           0.011935  0.015326  0.019076  0.029089   0.053222   
           eigen approx    0.012179  0.018851  0.014096  0.022622   0.040776   
           stl             0.036809  0.071125  0.135642  0.263596   0.513995   
           stl approx      0.007388  0.013634  0.007293  0.014253   0.026883   
           xsimd           0.017125  0.025050  0.040837  0.095256   0.142567   

Input Size                        128         256  
Layer Type Compute Method                          
conv1d     eigen            11.940000   96.349900  
           eigen approx      7.702350   62.194600  
           stl             123.585000  998.607000  
           stl approx       78.410500  620.229000  
           xsimd            21.329400  233.906000  
dense      eigen             0.117727    0.763305  
           eigen approx      0.116607    0.750825  
           stl               2.694920   13.509600  
           stl approx        2.715540   13.510800  
           xsimd             0.630264    3.669640  
gru        eigen             1.265960    4.609600  
           eigen approx      1.229340    4.485900  
           stl              16.430500   76.203300  
           stl approx       14.647700   70.024500  
           xsimd             3.911710   21.752500  
lstm       eigen             1.592930    6.128650  
           eigen approx      1.519310    5.926990  
           stl              22.749700  104.408000  

In [77]:
# Apply bold to the minimum values in each column within each Layer Type group
def bold_min_in_group(df):
    # Get the minimum of each column in the DataFrame
    min_vals = df.min()
    # Apply formatting: bold the minimum values
    return df.apply(lambda x: x.map(lambda v: f"\\textbf{{{v}}}" if v == min_vals[x.name] else v))

# Apply the function to each group of Layer Type
formatted_df = pivot_df.groupby(level='Layer Type').apply(bold_min_in_group)

# Convert the formatted DataFrame to LaTeX
latex_output = formatted_df.to_latex(escape=False, index=True)

print(latex_output)

\begin{tabular}{llllllllll}
\toprule
 &  & Input Size & 4 & 8 & 16 & 32 & 64 & 128 & 256 \\
Layer Type & Layer Type & Compute Method &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{conv1d} & \multirow[t]{5}{*}{conv1d} & eigen & \textbf{0.004805} & \textbf{0.0121261} & 0.064102 & 0.148811 & 1.407740 & 11.940000 & 96.349900 \\
 &  & eigen approx & 0.011008 & 0.017569 & \textbf{0.0493559} & \textbf{0.101118} & \textbf{0.908134} & \textbf{7.70235} & \textbf{62.1946} \\
 &  & stl & 0.007266 & 0.054377 & 0.653435 & 1.759890 & 15.142400 & 123.585000 & 998.607000 \\
 &  & stl approx & 0.012041 & 0.042484 & 0.435642 & 1.109390 & 9.020560 & 78.410500 & 620.229000 \\
 &  & xsimd & 0.010408 & 0.062329 & 0.402620 & 0.556938 & 2.716340 & 21.329400 & 233.906000 \\
\cline{1-10} \cline{2-10}
\multirow[t]{5}{*}{dense} & \multirow[t]{5}{*}{dense} & eigen & 0.008347 & 0.005731 & 0.004573 & 0.008962 & 0.032131 & 0.117727 & 0.763305 \\
 &  & eigen approx & 0.006982 & \textbf{0.00543071} & \textbf{0.0043

In [ ]:
# without bolding the minimum values
latex_output = pivot_df.to_latex(escape=False)

print(latex_output)

\begin{tabular}{lll}
\toprule
 &  & 0 \\
Layer Type & Compute Method &  \\
\midrule
\multirow[t]{5}{*}{conv1d} & eigen & ['\\textbf{0.004805}', 0.0121261, 0.0641018, 0.148811, 1.40774, 11.94, 96.3499] \\
 & eigen approx & ['\\textbf{0.0110078}', 0.0175686, 0.0493559, 0.101118, 0.908134, 7.70235, 62.1946] \\
 & stl & ['\\textbf{0.007266}', 0.0543769, 0.653435, 1.75989, 15.1424, 123.585, 998.607] \\
 & stl approx & ['\\textbf{0.012041}', 0.0424836, 0.435642, 1.10939, 9.02056, 78.4105, 620.229] \\
 & xsimd & ['\\textbf{0.0104075}', 0.0623288, 0.40262, 0.556938, 2.71634, 21.3294, 233.906] \\
\cline{1-3}
\multirow[t]{5}{*}{dense} & eigen & [0.00834742, 0.00573096, '\\textbf{0.0045725}', 0.008962, 0.0321311, 0.117727, 0.763305] \\
 & eigen approx & [0.00698158, 0.00543071, '\\textbf{0.00435225}', 0.00886629, 0.0316397, 0.116607, 0.750825] \\
 & stl & ['\\textbf{0.00516908}', 0.00834617, 0.0190953, 0.0898073, 0.49909, 2.69492, 13.5096] \\
 & stl approx & ['\\textbf{0.00515608}', 0.0080745, 0.